# Cleaning an Analysing data

Goal: the data needs to be clean so that it ca be used for the next steps
Requirements:
- Remove “U-bahn”
- Add “n” to the end of the location if it’s missing (Gesundbrunnen, Neukoeln, Tiergarten, etc. )
- timestamp (hh:mm:ss) - change e.g. "Mitternacht" and "After Mitternacht"
- date format (yyyy-mm-dd)
- Modify the last three unrelated crime : they can be deleted if A. it's a sole incident (not combined with other crimes) AND B. the number of the crime is relatively minor



In [356]:
# load csv into df
import pandas as pd
import re
import numpy as np
import os
from datetime import datetime
from datetime import timedelta
from time import strftime

In [10]:
folder_path = '/Users/ellenlee/code/hclpush/finding-conan/raw-data/structured-data/csv-output'
data = []
for file in os.listdir(folder_path):
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        data.append(df)
concatenated_df = pd.concat(data, ignore_index=True)
cur_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M")
output_path = f'/Users/ellenlee/code/hclpush/finding-conan/raw-data/structured-data/concacted_labeled_cases_{cur_datetime}.csv'
concatenated_df.to_csv(output_path, index=False)


In [17]:
df_original = pd.read_csv(output_path)

In [18]:
df = df_original.copy()

In [19]:
df.head(4)

,unique_case_id,official_case_id,type_of_crime,location,year,date,time,victim_sex,offender_sex,number_of_victims,number_of_offenders
0,000001_1167412,1167412,Property Damage,"Blankenburger Pflasterweg, Heinersdorfer Straße",2022,14.01,2.15 Uhr,"Female, Male",Male,3,1
1,000002_1167410,1167410,Homicide,Kühnemannstraße,2022,2.02,06.00 Uhr,Male,Male,1,1
2,000003_1167332,1167332,"General Assault, Verbal Abuse, Property Damage...",Rathausstraße,2022,14.01,1.15 Uhr,Male,Male,1,3
3,000004_1167270,1167270,"Property Crime, General Assault","Gropiusstadt, Neukölln",2022,20.06,Unknown,Male,Male,1,2


In [20]:
df.shape

(2405, 11)

In [14]:
# Number of unique_case_id before cleaning = 2303
df['unique_case_id'].nunique()

2303

In [21]:
crime_columns = ['Homicide', 'Hate Crime - Disability', 'Hate Crime - Gender', 'Hate Crime - Gender Identity', 'Hate Crime - Religious',
                  'Hate Crime - Sexual orientation', 'Hate Crime - Racial/Ethnicity', 'Hate Crime - Ethnicity', 'Verbal Abuse/Verbal Assault',
                  'Property Damage', 'Drug Offenses', 'General Assault', 'Sexual Assault', 'Sexual Harassment', 'Property Crimes',
                  'Domestic Violence', 'Missing Person', 'Traffic Incident', 'General Assault', 'Unclassified']

In [22]:
for index, row in df.iterrows():
    types_of_crime = str(row['type_of_crime']).split(", ")
    for column in crime_columns:
        df.loc[index, column] = 0
    for crime in types_of_crime:
        if crime in crime_columns:
            df.loc[index, crime] = 1
# df.to_csv('concatenated.csv', index=False)

In [24]:
df.head()

,unique_case_id,official_case_id,type_of_crime,location,year,date,time,victim_sex,offender_sex,number_of_victims,...,Property Damage,Drug Offenses,General Assault,Sexual Assault,Sexual Harassment,Property Crimes,Domestic Violence,Missing Person,Traffic Incident,Unclassified
0,000001_1167412,1167412,Property Damage,"Blankenburger Pflasterweg, Heinersdorfer Straße",2022,14.01,2.15 Uhr,"Female, Male",Male,3,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,000002_1167410,1167410,Homicide,Kühnemannstraße,2022,2.02,06.00 Uhr,Male,Male,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,000003_1167332,1167332,"General Assault, Verbal Abuse, Property Damage...",Rathausstraße,2022,14.01,1.15 Uhr,Male,Male,1,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,000004_1167270,1167270,"Property Crime, General Assault","Gropiusstadt, Neukölln",2022,20.06,Unknown,Male,Male,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,000005_1167075,1167075,Property Crime,"Hauptstraße, Tempelhof-Schöneberg",2022,13.01,2.40 Uhr,Unknown,Male,Unknown,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Clean the data 

In [277]:
# Create a new DataFrame to store the cleaned data
# cleaned_df = pd.DataFrame()
cleaned_df = df.copy()

In [278]:
cleaned_df

,unique_case_id,official_case_id,type_of_crime,location,year,date,time,victim_sex,offender_sex,number_of_victims,...,Property Damage,Drug Offenses,General Assault,Sexual Assault,Sexual Harassment,Property Crimes,Domestic Violence,Missing Person,Traffic Incident,Unclassified
0,000001_1167412,1167412,Property Damage,"Blankenburger Pflasterweg, Heinersdorfer Straße",2022,14.01,2.15 Uhr,"Female, Male",Male,3,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,000002_1167410,1167410,Homicide,Kühnemannstraße,2022,2.02,06.00 Uhr,Male,Male,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,000003_1167332,1167332,"General Assault, Verbal Abuse, Property Damage...",Rathausstraße,2022,14.01,1.15 Uhr,Male,Male,1,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,000004_1167270,1167270,"Property Crime, General Assault","Gropiusstadt, Neukölln",2022,20.06,Unknown,Male,Male,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,000005_1167075,1167075,Property Crime,"Hauptstraße, Tempelhof-Schöneberg",2022,13.01,2.40 Uhr,Unknown,Male,Unknown,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2400,000007_1327301,1327301,"Sexual Assault, Verbal Abuse",Lichtenberg,2023,23.05,Unknow,Unknow,Male,Unknow,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2401,000008_1327284,1327284,"General Assault, Verbal Abuse, Hate Crime Racial","Max-Josef-Metzger-Platz, Wedding",2023,23.05,18.30 Uhr,Male and Female,Male,7,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2402,000009_1327174,1327174,"General Assault, Verbal Abuse, Property Crimes...",Steglitz-Zehlendorf,2023,23.05,12.30 Uhr,Male,Male,2,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2403,000010_1327172,1327172,"General Assault, Property Damage","Dolgenseestraße, Lichtenberg",2023,23.05,13 Uhr,Male and Female,Unknow,40,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [279]:
# Drop duplicates
cleaned_df.drop_duplicates(subset=['official_case_id'], keep='last', inplace=True)
cleaned_df.shape # We have 2301 unqique rows/cases

(2301, 30)

In [280]:
# Combine' Hate Crime - Racial/Ethnicity' and 'Hate Crime - Ethnicity'
conditions = [(cleaned_df['Hate Crime - Racial/Ethnicity'] == 0) & (cleaned_df['Hate Crime - Ethnicity'] == 0),
              (cleaned_df['Hate Crime - Racial/Ethnicity'] == 1) & (cleaned_df['Hate Crime - Ethnicity'] == 0),
              (cleaned_df['Hate Crime - Racial/Ethnicity'] == 0) & (cleaned_df['Hate Crime - Ethnicity'] == 1),
              (cleaned_df['Hate Crime - Racial/Ethnicity'] == 1) & (cleaned_df['Hate Crime - Ethnicity'] == 1)]
choices = [0, 1, 1, 1]
cleaned_df['updated_re'] = np.select(conditions, choices)

In [281]:
# Quality check
cleaned_df.loc[(cleaned_df['Hate Crime - Racial/Ethnicity'] == 1) & (cleaned_df['Hate Crime - Ethnicity'] == 0)].head() 

,unique_case_id,official_case_id,type_of_crime,location,year,date,time,victim_sex,offender_sex,number_of_victims,...,Drug Offenses,General Assault,Sexual Assault,Sexual Harassment,Property Crimes,Domestic Violence,Missing Person,Traffic Incident,Unclassified,updated_re
159,000013_1234844,1234844,"Property Damage, Hate Crime - Racial/Ethnicity","Köllnische Straße, Rudower Straße, Niederschön...",2022,11.08 - 13.08,11.30 Uhr - 21.00 Uhr,Unknown,Unknown,79,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
191,000011_1177075,1177075,"Verbal Abuse, Hate Crime - Racial/Ethnicity",Alt-Moabit Straße,2022,16.02,14.20 Uhr,Female,"Female, Male",1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
224,000012_1198794,1198794,"General Assault, Verbal Abuse, Hate Crime - Ra...",U-Bahnhof Heinrich-Heine-Straße,2022,23.04,20.10 Uhr,Male,Male,1,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
289,000013_1261936,1261936,"General Assault, Verbal Abuse, Hate Crime - Ra...","Hobrechtsfelder Chaussee, Buch",2022,6.11,13.30 Uhr,Male,Male,2,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
318,000042_1259714,1259714,"General Assault, Verbal Abuse, Hate Crime - Ra...",Tempelhof-Schöneberg,2022,29.1,22.40 Uhr,Female,Male,1,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [282]:
# Replace the original column
cleaned_df = cleaned_df.drop(columns=['Hate Crime - Racial/Ethnicity', 'Hate Crime - Ethnicity'])
cleaned_df['Hate Crime - Racial/Ethnicity'] = cleaned_df['updated_re']
# cleaned_df.loc[cleaned_df['updated_re'] ==0] # Quality check

In [283]:
cleaned_df = cleaned_df.drop(columns=['updated_re'])

In [285]:
cleaned_df.columns

Index(['unique_case_id', 'official_case_id', 'type_of_crime', 'location',
       'year', 'date', 'time', 'victim_sex', 'offender_sex',
       'number_of_victims', 'number_of_offenders', 'Homicide',
       'Hate Crime - Disability', 'Hate Crime - Gender',
       'Hate Crime - Gender Identity', 'Hate Crime - Religious',
       'Hate Crime - Sexual orientation', 'Verbal Abuse/Verbal Assault',
       'Property Damage', 'Drug Offenses', 'General Assault', 'Sexual Assault',
       'Sexual Harassment', 'Property Crimes', 'Domestic Violence',
       'Missing Person', 'Traffic Incident', 'Unclassified',
       'Hate Crime - Racial/Ethnicity'],
      dtype='object')

In [286]:
# Task 1: Convert 'Mitternacht' to '00:00'
cleaned_df['time'] = df['time'].replace('Mitternacht', '00.00')

In [287]:
# Task 2: Convert 'kurz nach Mitternacht' to '00:05'
cleaned_df['time'] = df['time'].replace('kurz nach Mitternacht', '00.05')

# Task 3: Convert 'Nachmittag' to '15:00'
cleaned_df['time'] = cleaned_df['time'].replace('Nachmittag', '15.00')

# Task 3: Convert 'Nacht' to '00:00', 'Mitta' to 15.00, 'Nach' zu 00.00 and 'Vormi' zu 11.00
cleaned_df['time'] = cleaned_df['time'].replace('Nacht', '00.00')
cleaned_df['time'] = cleaned_df['time'].replace('Mitta', '15.00')
cleaned_df['time'] = cleaned_df['time'].replace('Nach', '00.00')
cleaned_df['time'] = cleaned_df['time'].replace('Vormi', '11.00')

# Task 4: Remove the second time in a timespan
cleaned_df['time'] = cleaned_df['time'].apply(lambda x: x.split('-')[0].strip() if isinstance(x, str) and '-' in x else x)

In [288]:
# Task 5: Remove 'Uhr' from the time values
cleaned_df['time'] = cleaned_df['time'].str.replace(' Uhr', '')

In [289]:
# Task 6 Add '.00' to times without minutes
cleaned_df['time'] = cleaned_df['time'].apply(lambda x: x + '.00' if isinstance(x, str) and re.match(r'^\d+(:\d+)?$', x) else x)

In [290]:
# Task 7: Remove the second time in a timespan
cleaned_df['date'] = cleaned_df['date'].apply(lambda x: x.split('-')[0].strip() if isinstance(x, str) and '-' in x else x)
cleaned_df['date'] = cleaned_df['date'].apply(lambda x: x.split('&')[0].strip() if isinstance(x, str) and '&' in x else x)
cleaned_df['date'] = cleaned_df['date'].apply(lambda x: x.split(',')[0].strip() if isinstance(x, str) and ',' in x else x)

In [291]:
# Task 8: split and add 0 date
cleaned_df['date'] = cleaned_df['date'].apply(lambda x: ('0' + x) if len(x.split('.')[0]) < 2 else (x.split('.')[0]+'.0'+x.split('.')[1]) if len(x.split('.')[1]) < 2 else ('0'+x.split('.')[0]+'.0'+x.split('.')[1]) if (len(x.split('.')[0]) < 2 and len(x.split('.')[1]) < 2) else x)

In [292]:
# Task 9: split and add 0 tim3
cleaned_df['time'] = cleaned_df['time'].apply(lambda x: ('0' + x) if len(x.split('.')) > 1 and len(x.split('.')[0]) < 2 else (x.split('.')[0]+'.0'+x.split('.')[1]) if len(x.split('.')) > 1 and len(x.split('.')[1]) < 2 else ('0'+x.split('.')[0]+'.0'+x.split('.')[1]) if len(x.split('.')) > 1 and len(x.split('.')[0]) < 2 and len(x.split('.')[1]) < 2 else x)



In [293]:
# Check status
cleaned_df['unique_case_id'].nunique()
cleaned_df.head(10)

,unique_case_id,official_case_id,type_of_crime,location,year,date,time,victim_sex,offender_sex,number_of_victims,...,Drug Offenses,General Assault,Sexual Assault,Sexual Harassment,Property Crimes,Domestic Violence,Missing Person,Traffic Incident,Unclassified,Hate Crime - Racial/Ethnicity
0,000001_1167412,1167412,Property Damage,"Blankenburger Pflasterweg, Heinersdorfer Straße",2022,14.01,02.15,"Female, Male",Male,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,000002_1167410,1167410,Homicide,Kühnemannstraße,2022,02.02,06.00,Male,Male,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,000003_1167332,1167332,"General Assault, Verbal Abuse, Property Damage...",Rathausstraße,2022,14.01,01.15,Male,Male,1,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,000004_1167270,1167270,"Property Crime, General Assault","Gropiusstadt, Neukölln",2022,20.06,Unknown,Male,Male,1,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,000005_1167075,1167075,Property Crime,"Hauptstraße, Tempelhof-Schöneberg",2022,13.01,02.40,Unknown,Male,Unknown,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5,000001_1318967,1318967,"General Assault, Property Crime, Verbal Abuse","Erwin-Barth-Platz, Charlottenburg-Wilmersdorf",2023,29.04,13.40,Male,Male,1,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
6,000002_1318965,1318965,"Property Damage, General Assault",Seydelstraße bis zur Alte-Jakob-Straße,2023,29.04,Mitternacht,Unknow,Male,Unknow,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
7,000003_1317835,1317835,"Property Crime, Hehlerei","Wilhelmstadt, Sandstraße",2023,27.04,22.00,Unknow,Male,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
8,000004_1317465,1317465,"Property Damage, Hate Crime - Gender Identity",Friedhof in Lichtenberg,2023,26.04,17.45,Transgender,Unknow,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9,000005_1317344,1317344,Property Damage,"Wedding, Togostraße",2023,26.04,23.00,Female,Unknow,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


## Timestamp and date

In [294]:
cleaned_df.head(5)

,unique_case_id,official_case_id,type_of_crime,location,year,date,time,victim_sex,offender_sex,number_of_victims,...,Drug Offenses,General Assault,Sexual Assault,Sexual Harassment,Property Crimes,Domestic Violence,Missing Person,Traffic Incident,Unclassified,Hate Crime - Racial/Ethnicity
0,000001_1167412,1167412,Property Damage,"Blankenburger Pflasterweg, Heinersdorfer Straße",2022,14.01,02.15,"Female, Male",Male,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,000002_1167410,1167410,Homicide,Kühnemannstraße,2022,02.02,06.00,Male,Male,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,000003_1167332,1167332,"General Assault, Verbal Abuse, Property Damage...",Rathausstraße,2022,14.01,01.15,Male,Male,1,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,000004_1167270,1167270,"Property Crime, General Assault","Gropiusstadt, Neukölln",2022,20.06,Unknown,Male,Male,1,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,000005_1167075,1167075,Property Crime,"Hauptstraße, Tempelhof-Schöneberg",2022,13.01,02.40,Unknown,Male,Unknown,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [295]:
# Replace "." with "-" in the 'date' column
cleaned_df['date'] = cleaned_df['date'].str.replace('.', '-')

# Create datetime column
cleaned_df['datetime'] = pd.to_datetime(cleaned_df.year + '-' + 
                                        cleaned_df.date + ' ' +
                                        cleaned_df.time, 
                                        format='%Y-%d-%m %H.%M',
                                        errors='coerce' # replace any invalid dates with NaT, but be aware some have date
                                       )
# Create date column
cleaned_df['year_date'] = pd.to_datetime(cleaned_df.year + '-' + 
                                        cleaned_df.date,
                                        format='%Y-%d-%m', 
                                        errors='coerce' # replace any invalid dates with NaT, but be aware some have date
                                       )


In [296]:
cleaned_df.head()

,unique_case_id,official_case_id,type_of_crime,location,year,date,time,victim_sex,offender_sex,number_of_victims,...,Sexual Assault,Sexual Harassment,Property Crimes,Domestic Violence,Missing Person,Traffic Incident,Unclassified,Hate Crime - Racial/Ethnicity,datetime,year_date
0,000001_1167412,1167412,Property Damage,"Blankenburger Pflasterweg, Heinersdorfer Straße",2022,14-01,02.15,"Female, Male",Male,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2022-01-14 02:15:00,2022-01-14
1,000002_1167410,1167410,Homicide,Kühnemannstraße,2022,02-02,06.00,Male,Male,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2022-02-02 06:00:00,2022-02-02
2,000003_1167332,1167332,"General Assault, Verbal Abuse, Property Damage...",Rathausstraße,2022,14-01,01.15,Male,Male,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2022-01-14 01:15:00,2022-01-14
3,000004_1167270,1167270,"Property Crime, General Assault","Gropiusstadt, Neukölln",2022,20-06,Unknown,Male,Male,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NaT,2022-06-20
4,000005_1167075,1167075,Property Crime,"Hauptstraße, Tempelhof-Schöneberg",2022,13-01,02.40,Unknown,Male,Unknown,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2022-01-13 02:40:00,2022-01-13


In [297]:
# Get number of NA for datetime (some might have date data)
cleaned_df['datetime'].isna().sum()

219

# Add the missing "n"

In [298]:
# Examine whether n is missing
print('Number of Gesundbrunne? ' + f"{cleaned_df['location'].str.contains('Gesundbrunne$', regex=True).sum()}")
print('Number of Neuköll? ' + f"{cleaned_df['location'].str.contains('Neuköll$', regex=True).sum()}")
print('Number of Tiergarte? ' + f"{cleaned_df['location'].str.contains('Tiergarte$', regex=True).sum()}")
print('Number of Berli? ' + f"{cleaned_df['location'].str.contains('Berli$', regex=True).sum()}")

Number of Gesundbrunne? 3
Number of Neuköll? 7
Number of Tiergarte? 3
Number of Berli? 2


In [299]:
import re
# List of words to modify
words_to_modify = ['Gesundbrunne', 'Neuköll', 'Tiergarte', 'Berli']

# Define the regular expression pattern
pattern = r'\b(' + '|'.join(words_to_modify) + r')\b'

# Update the values in the 'location' column using regular expressions
cleaned_df['location'] = cleaned_df['location'].apply(lambda x: re.sub(pattern, r'\1n', x))

# Print the updated dataframe
cleaned_df.head(5)

,unique_case_id,official_case_id,type_of_crime,location,year,date,time,victim_sex,offender_sex,number_of_victims,...,Sexual Assault,Sexual Harassment,Property Crimes,Domestic Violence,Missing Person,Traffic Incident,Unclassified,Hate Crime - Racial/Ethnicity,datetime,year_date
0,000001_1167412,1167412,Property Damage,"Blankenburger Pflasterweg, Heinersdorfer Straße",2022,14-01,02.15,"Female, Male",Male,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2022-01-14 02:15:00,2022-01-14
1,000002_1167410,1167410,Homicide,Kühnemannstraße,2022,02-02,06.00,Male,Male,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2022-02-02 06:00:00,2022-02-02
2,000003_1167332,1167332,"General Assault, Verbal Abuse, Property Damage...",Rathausstraße,2022,14-01,01.15,Male,Male,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2022-01-14 01:15:00,2022-01-14
3,000004_1167270,1167270,"Property Crime, General Assault","Gropiusstadt, Neukölln",2022,20-06,Unknown,Male,Male,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NaT,2022-06-20
4,000005_1167075,1167075,Property Crime,"Hauptstraße, Tempelhof-Schöneberg",2022,13-01,02.40,Unknown,Male,Unknown,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2022-01-13 02:40:00,2022-01-13


In [300]:
# Examine whether still exists
print('Number of Gesundbrunne? ' + f"{cleaned_df['location'].str.contains('Gesundbrunne$', regex=True).sum()}")
print('Number of Neuköll? ' + f"{cleaned_df['location'].str.contains('Neuköll$', regex=True).sum()}")
print('Number of Tiergarte? ' + f"{cleaned_df['location'].str.contains('Tiergarte$', regex=True).sum()}")
print('Number of Berli? ' + f"{cleaned_df['location'].str.contains('Berli$', regex=True).sum()}")

Number of Gesundbrunne? 0
Number of Neuköll? 0
Number of Tiergarte? 0
Number of Berli? 0


# Remove U-Bahnhof

In [301]:
# Remove 'U-Bahnhof' from values in the 'location' column
cleaned_df['location'] = cleaned_df['location'].str.replace('U-Bahnhof', '')
cleaned_df['location'] = cleaned_df['location'].str.replace('S-Bahnhof', '')
cleaned_df['location'] = cleaned_df['location'].str.replace('U-Bahn', '')
cleaned_df['location'] = cleaned_df['location'].str.replace('S-Bahn', '')

In [302]:
# Print the updated dataframe
cleaned_df.head(20)

,unique_case_id,official_case_id,type_of_crime,location,year,date,time,victim_sex,offender_sex,number_of_victims,...,Sexual Assault,Sexual Harassment,Property Crimes,Domestic Violence,Missing Person,Traffic Incident,Unclassified,Hate Crime - Racial/Ethnicity,datetime,year_date
0,000001_1167412,1167412,Property Damage,"Blankenburger Pflasterweg, Heinersdorfer Straße",2022,14-01,02.15,"Female, Male",Male,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2022-01-14 02:15:00,2022-01-14
1,000002_1167410,1167410,Homicide,Kühnemannstraße,2022,02-02,06.00,Male,Male,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2022-02-02 06:00:00,2022-02-02
2,000003_1167332,1167332,"General Assault, Verbal Abuse, Property Damage...",Rathausstraße,2022,14-01,01.15,Male,Male,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2022-01-14 01:15:00,2022-01-14
3,000004_1167270,1167270,"Property Crime, General Assault","Gropiusstadt, Neukölln",2022,20-06,Unknown,Male,Male,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NaT,2022-06-20
4,000005_1167075,1167075,Property Crime,"Hauptstraße, Tempelhof-Schöneberg",2022,13-01,02.40,Unknown,Male,Unknown,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2022-01-13 02:40:00,2022-01-13
5,000001_1318967,1318967,"General Assault, Property Crime, Verbal Abuse","Erwin-Barth-Platz, Charlottenburg-Wilmersdorf",2023,29-04,13.40,Male,Male,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2023-04-29 13:40:00,2023-04-29
6,000002_1318965,1318965,"Property Damage, General Assault",Seydelstraße bis zur Alte-Jakob-Straße,2023,29-04,Mitternacht,Unknow,Male,Unknow,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NaT,2023-04-29
7,000003_1317835,1317835,"Property Crime, Hehlerei","Wilhelmstadt, Sandstraße",2023,27-04,22.00,Unknow,Male,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2023-04-27 22:00:00,2023-04-27
8,000004_1317465,1317465,"Property Damage, Hate Crime - Gender Identity",Friedhof in Lichtenberg,2023,26-04,17.45,Transgender,Unknow,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2023-04-26 17:45:00,2023-04-26
9,000005_1317344,1317344,Property Damage,"Wedding, Togostraße",2023,26-04,23.00,Female,Unknow,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2023-04-26 23:00:00,2023-04-26


# Save Cleaned data in new CSV

In [303]:
# Copy Dataframe
final_df = cleaned_df.copy()

In [304]:
final_df.head(5)

,unique_case_id,official_case_id,type_of_crime,location,year,date,time,victim_sex,offender_sex,number_of_victims,...,Sexual Assault,Sexual Harassment,Property Crimes,Domestic Violence,Missing Person,Traffic Incident,Unclassified,Hate Crime - Racial/Ethnicity,datetime,year_date
0,000001_1167412,1167412,Property Damage,"Blankenburger Pflasterweg, Heinersdorfer Straße",2022,14-01,02.15,"Female, Male",Male,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2022-01-14 02:15:00,2022-01-14
1,000002_1167410,1167410,Homicide,Kühnemannstraße,2022,02-02,06.00,Male,Male,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2022-02-02 06:00:00,2022-02-02
2,000003_1167332,1167332,"General Assault, Verbal Abuse, Property Damage...",Rathausstraße,2022,14-01,01.15,Male,Male,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2022-01-14 01:15:00,2022-01-14
3,000004_1167270,1167270,"Property Crime, General Assault","Gropiusstadt, Neukölln",2022,20-06,Unknown,Male,Male,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NaT,2022-06-20
4,000005_1167075,1167075,Property Crime,"Hauptstraße, Tempelhof-Schöneberg",2022,13-01,02.40,Unknown,Male,Unknown,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2022-01-13 02:40:00,2022-01-13


In [305]:
# drop not needed columns: unique_case_id
final_df = final_df.drop(columns=['unique_case_id', 'year', 'date'])
final_df.head()

,official_case_id,type_of_crime,location,time,victim_sex,offender_sex,number_of_victims,number_of_offenders,Homicide,Hate Crime - Disability,...,Sexual Assault,Sexual Harassment,Property Crimes,Domestic Violence,Missing Person,Traffic Incident,Unclassified,Hate Crime - Racial/Ethnicity,datetime,year_date
0,1167412,Property Damage,"Blankenburger Pflasterweg, Heinersdorfer Straße",02.15,"Female, Male",Male,3,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2022-01-14 02:15:00,2022-01-14
1,1167410,Homicide,Kühnemannstraße,06.00,Male,Male,1,1,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2022-02-02 06:00:00,2022-02-02
2,1167332,"General Assault, Verbal Abuse, Property Damage...",Rathausstraße,01.15,Male,Male,1,3,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2022-01-14 01:15:00,2022-01-14
3,1167270,"Property Crime, General Assault","Gropiusstadt, Neukölln",Unknown,Male,Male,1,2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,NaT,2022-06-20
4,1167075,Property Crime,"Hauptstraße, Tempelhof-Schöneberg",02.40,Unknown,Male,Unknown,1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2022-01-13 02:40:00,2022-01-13


In [309]:
final_df.columns = map(str.lower, final_df.columns)
final_df.columns = [col.replace(' ', '_').replace('_-_', '-') for col in final_df.columns]

In [310]:
final_df.columns

Index(['official_case_id', 'type_of_crime', 'location', 'time', 'victim_sex',
       'offender_sex', 'number_of_victims', 'number_of_offenders', 'homicide',
       'hate_crime-disability', 'hate_crime-gender',
       'hate_crime-gender_identity', 'hate_crime-religious',
       'hate_crime-sexual_orientation', 'verbal_abuse/verbal_assault',
       'property_damage', 'drug_offenses', 'general_assault', 'sexual_assault',
       'sexual_harassment', 'property_crimes', 'domestic_violence',
       'missing_person', 'traffic_incident', 'unclassified',
       'hate_crime-racial/ethnicity', 'datetime', 'year_date'],
      dtype='object')

In [311]:
arranged_col_lst = [
    'official_case_id', 'type_of_crime', 'location', 
    'datetime', 'year_date',
    'time', 
    'victim_sex', 'offender_sex', 'number_of_victims', 'number_of_offenders', 'homicide',
   'hate_crime-disability', 'hate_crime-gender',
   'hate_crime-gender_identity', 'hate_crime-religious',
   'hate_crime-sexual_orientation', 'hate_crime-racial/ethnicity',
    'verbal_abuse/verbal_assault',
   'property_damage', 'drug_offenses', 'general_assault', 'sexual_assault',
   'sexual_harassment', 'property_crimes', 
#         'domestic_violence','missing_person', 'traffic_incident', 
        'unclassified'
       
                    ]

final_df = final_df[arranged_col_lst] 

In [317]:
final_df['verbal_abuse/verbal_assault'].sum()

0.0

In [436]:
final_df

,official_case_id,type_of_crime,location,datetime,year_date,time,victim_sex,offender_sex,number_of_victims,number_of_offenders,...,hate_crime-sexual_orientation,hate_crime-racial/ethnicity,verbal_abuse/verbal_assault,property_damage,drug_offenses,general_assault,sexual_assault,sexual_harassment,property_crimes,unclassified
0,1167412,Property Damage,"Blankenburger Pflasterweg, Heinersdorfer Straße",2022-01-14 02:15:00,2022-01-14,02.15,"Female, Male",Male,3,1,...,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1167410,Homicide,Kühnemannstraße,2022-02-02 06:00:00,2022-02-02,06.00,Male,Male,1,1,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1167332,"General Assault, Verbal Abuse, Property Damage...",Rathausstraße,2022-01-14 01:15:00,2022-01-14,01.15,Male,Male,1,3,...,0.0,0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
3,1167270,"Property Crime, General Assault","Gropiusstadt, Neukölln",NaT,2022-06-20,Unknown,Male,Male,1,2,...,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1167075,Property Crime,"Hauptstraße, Tempelhof-Schöneberg",2022-01-13 02:40:00,2022-01-13,02.40,Unknown,Male,Unknown,1,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2400,1327301,"Sexual Assault, Verbal Abuse",Lichtenberg,NaT,2023-05-23,Unknow,Unknow,Male,Unknow,1,...,0.0,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2401,1327284,"General Assault, Verbal Abuse, Hate Crime Racial","Max-Josef-Metzger-Platz, Wedding",2023-05-23 18:30:00,2023-05-23,18.30,Male and Female,Male,7,1,...,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2402,1327174,"General Assault, Verbal Abuse, Property Crimes...",Steglitz-Zehlendorf,2023-05-23 12:30:00,2023-05-23,12.30,Male,Male,2,1,...,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2403,1327172,"General Assault, Property Damage","Dolgenseestraße, Lichtenberg",2023-05-23 13:00:00,2023-05-23,13.00,Male and Female,Unknow,40,Unknow,...,0.0,0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [323]:
cur_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M")
final_df.to_csv(f"/Users/ellenlee/code/hclpush/finding-conan/raw-data/structured-data/final-crime-data_{cur_datetime}.csv")


## Adding timeslot

In [454]:
wip_df = final_df.copy()

In [455]:
# How many Berlinweit do we have
wip_df.loc[wip_df['location'] == 'Berlinweit'].shape[0] 

35

In [456]:
# How many Berlin do we have
wip_df.loc[wip_df['location'] == 'Berlin'].shape[0]

1

In [457]:
wip_df = wip_df.loc[wip_df['location'] != 'Berlinweit']

In [458]:
# How many Berlinweit do we have
wip_df.loc[wip_df['location'] == 'Berlinweit'].shape[0] 

0

In [459]:
# How many Berlin do we have
wip_df = wip_df.loc[wip_df['location'] != 'Berlin']

In [460]:
# How many Berlinweit do we have
wip_df.loc[wip_df['location'] == 'Berlinweit'].shape[0] 

0

In [461]:
wip_df.shape

(2265, 25)

In [462]:
# Sort df
wip_df = wip_df.sort_values(by=['datetime'])

# Remove data outside 2021/9/13 and 2022/5/24
start_time = datetime.datetime(2021, 9, 13, 0, 0)  # Starting datetime
end_time = datetime.datetime(2023, 5, 25, 0, 0)  # Ending datetime
wip_df = wip_df.loc[(wip_df['datetime'] >= start_time) & (wip_df['datetime'] <= end_time)].reset_index(drop=True)
wip_df.head()

,official_case_id,type_of_crime,location,datetime,year_date,time,victim_sex,offender_sex,number_of_victims,number_of_offenders,...,hate_crime-sexual_orientation,hate_crime-racial/ethnicity,verbal_abuse/verbal_assault,property_damage,drug_offenses,general_assault,sexual_assault,sexual_harassment,property_crimes,unclassified
0,1125354,Property Damage,"Bugenhagenstraße, Moabit",2021-09-13 00:30:00,2021-09-13,00.30,Unknown,Male,0,3,...,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1125574,Property Damage,"Mierendorffplatz, Charlottenburg-Wilmersdorf",2021-09-13 05:00:00,2021-09-13,05.00,Unknown,Unknown,0,Unknown,...,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1125707,"General Assault, Property Crime","Parkanlage am Brunnenplatz, Gropiusstraße, Bad...",2021-09-13 08:30:00,2021-09-13,08.30,Male,Male,1,4,...,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1125352,Property Damage,Platz der Republik,2021-09-13 13:40:00,2021-09-13,13.40,Unknown,Male,Unknown,1,...,0.0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1125567,"General Assault, Resisting Arrest",Axel-Springer-Straße,2021-09-13 17:15:00,2021-09-13,17.15,Male,Male,1,2,...,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1939,1327165,"Verbal Abuse, Hate Crime Sexual Orientatio","Adlershof, Treptow-Köpenick",2023-05-23 19:00:00,2023-05-23,19.00,Female,Male,1,1,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1940,1327344,"General Assault, Verbal Abuse, Robbery",Marzahn-Hellersdorf,2023-05-23 23:45:00,2023-05-23,23.45,Male,Male,1,1-8,...,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1941,1327768,"General Assault, Verbal Abuse, Hate Crime Gend...","Alexanderstraße, Mitte",2023-05-24 17:00:00,2023-05-24,17.00,Female,Male,1,1,...,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1942,1327762,"General Assault, Verbal Abuse, Hate Crime Reli...",Marzahn-Hellersdorf,2023-05-24 17:45:00,2023-05-24,17.45,Male,Male,1,2,...,0.0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [482]:
def get_time_slot(tval):
    """ Return the label associated with the timestamp """
    labels = ['Midnight', 'Early Morning', 'Morning', 'Afternoon', 'Evening', 'Night']
    slot_start = [(0, 0), (4, 0), (8, 0), (12, 0), (16, 0), (20, 0)]
    for lidx, tme in enumerate(slot_start):
        if tme[0] > tval.hour:
            return labels[lidx-1]
        elif tval.hour == tme[0] and tme[1] <= tval.minute:
            return labels[lidx]
    return labels[-1]  

In [483]:
wip_df['time_slot'] = wip_df.apply(lambda row: get_time_slot(row.datetime), axis=1)

In [484]:
wip_df[['datetime', 'time_slot']]

,datetime,time_slot
0,2021-09-13 00:30:00,Midnight
1,2021-09-13 05:00:00,Early Morning
2,2021-09-13 08:30:00,Morning
3,2021-09-13 13:40:00,Afternoon
4,2021-09-13 17:15:00,Evening
...,...,...
1939,2023-05-23 19:00:00,Evening
1940,2023-05-23 23:45:00,Night
1941,2023-05-24 17:00:00,Evening
1942,2023-05-24 17:45:00,Evening
